# VacationPy 

#### Note

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from google_config import gkey

### Store Part I results into DataFrame

In [2]:
df = pd.read_csv("cities.csv")
df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Punta Arenas,40,CL,1585187951,71,-53.15,-70.92,55.40,17.22
1,1,Rūjiena,0,LV,1585188143,87,57.90,25.33,32.32,8.84
2,2,Port Alfred,90,ZA,1585188060,59,-33.59,26.89,64.74,33.40
3,3,Dikson,100,RU,1585188067,97,73.51,80.55,21.78,22.70
4,4,Rikitea,100,PF,1585188058,76,-23.12,-134.97,78.84,5.35


### Humidity Heatmap

In [11]:
gmaps.configure(api_key=gkey)
# Customize window.
figure_layout = {
    'width':"720px",
    "height": "480px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout = figure_layout)
# Store locations
locations = df[["Lat", "Lng"]]
# Store humidity values
humidity_df = df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations=locations, weights=humidity_df, 
                                 dissipating=True, max_intensity=df["Humidity"].max(), point_radius=10)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='480px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [10]:
ideal_df = df.loc[(df["Max Temp"] > 60) & (df["Max Temp"] < 85) & 
                  (df["Wind Speed"] > 10) & (df["Cloudiness"] < 50) 
                 & (df["Humidity"] < 70)].reset_index()

### Hotel Map 

In [5]:
place_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
geo_url =  'https://maps.googleapis.com/maps/api/geocode/json'

In [6]:
hotel_name = []
hotel_city = []
hotel_country = []
hotel_lat = []
hotel_lng = []

for index, row in ideal_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    location = str(lat) + ", " + str(lng)
    params_place = {'key':gkey
             ,'location':location,
              "radius":5000,
              "type":"hotel",
              "keyword":"hotel"
             }
    params_geo = {'key':gkey
                  ,'address':location}
    try:
        response_place = requests.get(place_url, params=params_place).json()
        hotel_lat.append(response_place["results"][0]["geometry"]["location"]["lat"])
        hotel_lng.append(response_place["results"][0]["geometry"]["location"]["lng"])
        hotel_name.append(response_place["results"][0]["name"])

    except:
        hotel_lat.append(np.nan)
        hotel_lng.append(np.nan)
        hotel_name.append(np.nan)
    try:
        response_geo = requests.get(geo_url, params=params_geo).json()
        hotel_city.append(response_geo["results"][0]["address_components"][4]["long_name"])
    except:
        hotel_city.append(np.nan)
    try:
        response_geo = requests.get(geo_url, params=params_geo).json()
        hotel_country.append(response_geo["results"][0]["address_components"][5]["long_name"])
    except:
        hotel_country.append(np.nan)

In [7]:
hotel_dir = {"Hotel Name":hotel_name, "Lat":hotel_lat, 
             "Lng": hotel_lng, "Country":hotel_country,
             "City":hotel_city
            }
hotel_df = pd.DataFrame(hotel_dir).dropna()     
hotel_df = hotel_df[~hotel_df['Country'].str.contains('^\d+$')]
hotel_df

,Hotel Name,Lat,Lng,Country,City
3,OYO Hotel Miami Acapulco,16.857559,-99.895042,Mexico,Guerrero
4,Red Lion Hotel & Conference Center St. George,37.093510,-113.586869,United States,Utah
6,Mercure Alice Springs Resort,-23.702999,133.886497,Australia,Northern Territory
7,Mojo Hotel,-33.919248,18.385941,Western Cape,Cape Town
9,Courtyard by Marriott Maui Kahului Airport,20.888382,-156.449614,United States,Hawaii
14,Estepa Hotel,-39.029185,-67.576158,Argentina,Río Negro
17,"My Place Hotel - North Las Vegas, NV",36.241361,-115.124610,United States,Nevada
20,Hotel Boston,-34.722409,135.865463,Australia,South Australia
22,Hotel Laja Real,8.127362,-63.536800,Venezuela,Bolívar


In [13]:
gmaps.configure(api_key=gkey)
# Customize window.
figure_layout = {
    'width':"720px",
    "height": "480px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout = figure_layout)
# Store locations
locations = df[["Lat", "Lng"]]
# Store humidity values
humidity_df = df["Humidity"]
heat_layer = gmaps.heatmap_layer(locations=locations, weights=humidity_df, 
                                 dissipating=True, max_intensity=df["Humidity"].max(), point_radius=10)
fig.add_layer(heat_layer)
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

In [14]:
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='480px', margin='0 auto 0 auto', padding='1px', wi…